In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import StratifiedKFold, GroupKFold,  StratifiedGroupKFold, KFold
from IPython.display import display_html, display
import gezi
from gezi import tqdm

In [2]:
df = pd.read_csv("../input/feedback-prize-2021/train.csv")
cluster = pd.read_csv("../input/feedback-prize-2021/cluster.csv")
df = df.merge(cluster, on=['id'], how='left')

In [3]:
# 1015 1018 1028 1035 1046
# seed = 1046
# seed = 20211256
# seed = 1238923211
seed = 12768
ic(seed)
np.random.seed(seed)
dfx = df[['id', 'cluster']].drop_duplicates()
dfx['kfold'] = [np.random.randint(5) for _ in range(len(dfx))]
if 'kfold' in df.columns:
  df.drop('kfold', axis=1, inplace=True)
train = df.merge(dfx[["id", "kfold"]], on="id", how="left")
grp = train.groupby(["kfold", 'discourse_type'], as_index=False).count()
display(grp.pivot(index='discourse_type', columns='kfold', values='id').T)
display(dfx.groupby(["kfold", 'cluster'], as_index=False).count().pivot(index='cluster', columns='kfold', values='id').T)
assert len(train.groupby(["id", "kfold"], as_index=False).count()) == train["id"].nunique()
dfx.head()

ic| 4121716589.py:6 in <module>- seed: 12768


discourse_type,Claim,Concluding Statement,Counterclaim,Evidence,Lead,Position,Rebuttal
kfold,,,,,,,
0,10091,2670,1145,9094,1826,3062,863
1,10301,2749,1157,9354,1931,3176,854
2,9804,2709,1137,9054,1829,3087,852
3,9795,2648,1197,9016,1802,3019,886
4,10217,2729,1181,9184,1917,3075,882


cluster,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
kfold,,,,,,,,,,,,,,,
0,160,332,183,212,355,295,120,148,145,289,177,195,173,160,150
1,186,288,188,220,373,286,161,162,164,321,175,209,175,159,148
2,154,304,181,244,384,255,133,172,126,290,176,202,170,187,142
3,184,297,169,218,369,282,137,161,129,274,155,193,167,154,160
4,156,323,207,206,333,265,135,176,139,315,151,176,191,183,160


,id,cluster,kfold
0,423A1CA112E2,8,2
10,A8445CABFECE,8,4
14,6B4F7A0165B9,8,3
21,E05C7F5C1156,8,0
33,50B3435E475B,8,4


In [4]:
# K = 5
# SEED = 1111
# dfx = df[['id', 'cluster']].drop_duplicates()
# skf = StratifiedKFold(n_splits=K, random_state=SEED, shuffle=True)
# x = df.groupby('id')['discourse_type'].agg(list).reset_index()
# x['Rebuttal'] = df['discourse_type'].apply(lambda l: int('Rebuttal' in l))
# dfx = dfx.merge(x, on='id', how='left')
# splits = list(skf.split(X=dfx, y=dfx['Rebuttal']))

In [5]:
# folds = np.zeros(len(dfx), dtype=int)
# for i, (train_idx, val_idx) in enumerate(splits):
#   folds[val_idx] = i
# dfx['kfold'] = folds

In [6]:
# K = 5
# # SEED = 222
# # SEED = 12356
# # SEED = 102453
# SEED = 1203
# ic(SEED)
# dfx = df[['id', 'cluster']].drop_duplicates()
# df['Rebuttal'] = (df['discourse_type'] == 'Rebuttal').astype(int)
# df['Counterclaim'] = (df['discourse_type'] == 'Counterclaim').astype(int)
# df['Rare'] = df['discourse_type'].apply(lambda x: x in ['Rebuttal', 'Counterclaim']).astype(int)
# skf = StratifiedGroupKFold(n_splits=K, random_state=SEED, shuffle=True)
# splits = list(skf.split(X=df, y=df['Rare'], groups=df['id']))

In [7]:
# folds = np.zeros(len(df), dtype=int)
# for i, (train_idx, val_idx) in enumerate(splits):
#   folds[val_idx] = i
# df['kfold'] = folds

In [8]:
# ic(df.Rebuttal.sum() / 5, df.Counterclaim.sum() / 5)

In [9]:
# train = df.merge(dfx[["id", "cluster"]], on="id", how="left")
# grp = train.groupby(["kfold", 'discourse_type'], as_index=False).count()
# display(grp.pivot(index='discourse_type', columns='kfold', values='id').T)

In [10]:
# x = grp.pivot(index='discourse_type', columns='kfold', values='id').T

In [11]:
m = {}
# 1203
best_seed = None
best_a, best_b = 1000, 1000
def gen_folds(seed):
  global best_seed,best_a, best_b
  skf = StratifiedGroupKFold(n_splits=K, random_state=seed, shuffle=True)
  splits = list(skf.split(X=df, y=df['Rare'], groups=df['id']))
  folds = np.zeros(len(df), dtype=int)
  for i, (train_idx, val_idx) in enumerate(splits):
    folds[val_idx] = i
  df['kfold'] = folds
  train = df.merge(dfx[["id", "cluster"]], on="id", how="left")
  grp = train.groupby(["kfold", 'discourse_type'], as_index=False).count()
  x = grp.pivot(index='discourse_type', columns='kfold', values='id').T
  a = abs(x['Rebuttal'] - x['Rebuttal'].mean()).max()
  b = abs(x['Counterclaim'] - x['Counterclaim'].mean()).max()
  m[seed] = (a, b)
  if a < best_a and b < best_b:
    best_a = a
    best_b = b
    best_seed = seed
    ic(seed, best_a, best_b)

In [12]:
# for seed in tqdm(range(1024, 2048)):
#   gen_folds(seed)

In [14]:
# dfx = df[['id', 'cluster', 'kfold']].drop_duplicates()

In [15]:
display(dfx.groupby(["kfold", 'cluster'], as_index=False).count().pivot(index='cluster', columns='kfold', values='id').T)

cluster,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
kfold,,,,,,,,,,,,,,,
0,160,332,183,212,355,295,120,148,145,289,177,195,173,160,150
1,186,288,188,220,373,286,161,162,164,321,175,209,175,159,148
2,154,304,181,244,384,255,133,172,126,290,176,202,170,187,142
3,184,297,169,218,369,282,137,161,129,274,155,193,167,154,160
4,156,323,207,206,333,265,135,176,139,315,151,176,191,183,160


In [16]:
dfx[['id', 'cluster', 'kfold']].to_csv('../input/feedback-prize-2021/folds.csv')

In [17]:
dfx

,id,cluster,kfold
0,423A1CA112E2,8,2
10,A8445CABFECE,8,4
14,6B4F7A0165B9,8,3
21,E05C7F5C1156,8,0
33,50B3435E475B,8,4
...,...,...,...
144240,8F4B595CF9E7,1,2
144252,6B5809C83978,1,1
144261,408A7D3D2EEC,1,3
144270,AFEC37C2D43F,1,0


In [18]:
dfx.describe()

,cluster,kfold
count,15594.000000,15594.000000
mean,6.471784,1.992176
std,4.199462,1.412331
min,0.000000,0.000000
25%,3.000000,1.000000
50%,6.000000,2.000000
75%,10.000000,3.000000
max,14.000000,4.000000
